# Extracting text from TEI files

In [1]:
import glob
import os

import pandas as pd

from bs4 import BeautifulSoup

In [2]:
meta_file = '../metadata/DBNL-publication_author_metadata-english.tsv'

random_state = 743737

title_df = pd.read_csv(meta_file, sep='\t')
#sample_df = title_df.groupby('decade_end').sample(5, random_state=random_state)
title_ids = set(title_df.doc_id.unique())

In [3]:
len(title_ids)

2066

In [4]:
title_df

,doc_id,title,author_id,first_name,surname_prefix,surname,full_name,birth_year,death_year,year_of_publication
0,kist001leve01,"Het leven, gevoelens en zonderlinge reize van ...",kist001,Willem,NaN,Kist,"Kist, Willem",1758.0,1841.0,1800
1,stre001char01,"Charakters en lotgevallen van Adelson, Héloïse...",stre001,Naatje,van,Streek-Brinkman,"Streek-Brinkman, Naatje van",1760.0,1828.0,1804
2,daal002will01,Willem Hups. Eene anecdote uit de XVII eeuw; o...,daal002,Bruno,NaN,Daalberg,"Daalberg, Bruno",1758.0,1818.0,1805
3,bodd002leve01,Levensgeschiedenis van den vermaarden dichter,bodd002,Pieter,NaN,Boddaert jr.,"Boddaert jr., Pieter",1766.0,1805.0,1805
4,loos005hist01,Historie van Mejuffrouw Susanna Bronkhorst. De...,loos005,Adriaan,NaN,Loosjes,"Loosjes, Adriaan",1761.0,1818.0,1806
...,...,...,...,...,...,...,...,...,...,...
2061,merw017coff02,Coffy de negerslaaf,merw017,H.,te,Merwe,"Merwe, H. te",1896.0,1977.0,1966
2062,aard002stij01,Stijfkopje omnibus (onder pseudoniem Ankie Aal...,aard002,H.P.,van den,Aardweg,"Aardweg, H.P. van den",1899.0,1971.0,1970
2063,gras003daar01,"Daar ga je, Claudia!",gras003,Cok,NaN,Grashoff,"Grashoff, Cok",1928.0,2020.0,1970
2064,nijn001tolh02,"Tolhuis ""Het zwarte paard""",nijn001,H.J.,van,Nijnatten-Doffegnies,"Nijnatten-Doffegnies, H.J. van",1898.0,1990.0,1941


In [8]:
tei_dir = '../../../../Data/DBNL/xml/1800-2000/xml Proza 19e en 20e eeuw algemeen DBNL 20250122/'

tei_files = glob.glob(os.path.join(tei_dir, '*'))
tei_file_map = {tf.split('/')[-1].replace('_01.xml', ''): tf for tf in tei_files}
#tei_file_map.keys()

In [9]:
for ti_id in title_ids:
    if ti_id not in tei_file_map:
        print([tf_id for tf_id in tei_file_map if ti_id in tf_id])
        print(ti_id)

[]
schm001jipe01
[]
oors001twee02


In [10]:
tei_file_map = {tf: tei_file_map[tf] for tf in tei_file_map if tf in title_ids}
len(tei_file_map)

2064

In [13]:
soup_map = {}

for ti, ti_id in enumerate(tei_file_map):
    if ti_id not in tei_file_map:
        print(f"skipping {ti_id}")
        continue
    with open(tei_file_map[ti_id], 'rt') as fh:
        soup_map[ti_id] = BeautifulSoup(fh, features='xml')
    if (ti+1) > 1:
        break
        


In [14]:
sents = [sent for sent in doc.sents]
len(doc), len(sents[0])

selected_sents = filter_tokens(doc, 200, 200)
sum([len(sent) for sent in selected_sents])

NameError: name 'doc' is not defined

In [15]:
from bs4 import BeautifulSoup
from collections import Counter
from collections import defaultdict


item_freq = Counter()
ti_item_freq = defaultdict(Counter)
chapter_item_freq = defaultdict(Counter)
div_map = defaultdict(list)

for ti, ti_id in enumerate(soup_map):
    soup = soup_map[ti_id]
    body = soup.find('body')
    divs = soup.find_all('div')
    for div in divs:
        for item in div.attrs.items():
            key = '--'.join(item)
            div_map[key].append(div)
    chapters = [div for div in soup.find_all('div') if 'type' in div.attrs and div.attrs['type'] == 'chapter']
    item_freq.update(['--'.join(item) for div in body.find_all('div') for item in div.attrs.items()])
    ti_item_freq[ti_id].update(['--'.join(item) for div in body.find_all('div') for item in div.attrs.items()])
    for ci, chapter in enumerate(chapters):
        ch_id = f"{ti_id}-chapter-{ci:0>3}"
    
        chapter_item_freq[ch_id].update(['--'.join(item) for div in chapter.find_all('div') for item in div.attrs.items()])
    children = [child for child in body if child.name is not None]
    if (ti+1) > 500:
        break

item_freq

Counter({'type--chapter': 34, 'type--section': 5})

All `div` elements with attribute `ebook-type` or `wpg-type` are not part of the main prose text.

In [16]:
def extract_attr_items(div):
    return ['--'.join(item) for item in div.attrs.items()]


item_rel_freq = Counter()

for ti, ti_id in enumerate(soup_map):
    soup = soup_map[ti_id]
    body = soup.find('body')
    divs = soup.find_all('div')
    for parent_div in divs:
        parent_items = extract_attr_items(parent_div)
        for child_div in parent_div.find_all('div'):
            child_items = extract_attr_items(child_div)
            item_rel_freq.update([(parent_item, child_item) for parent_item in parent_items for child_item in child_items])

In [17]:
for item in sorted(item_rel_freq):
    print(item, item_rel_freq[item])

('type--chapter', 'type--section') 5


### Non-Dutch elements

In [18]:
for key in div_map:
    #print(key)
    #if key != 'type--chapter':
    #    continue
    #print(Counter([item for div in div_map[key] for item in div.attrs.items()]))
    for div in div_map[key]:
        if 'type' in div.attrs and div.attrs['type'] == 'chapter':
            if 'ebook-type' in div.attrs:
                print(div)
        if 'lang' in div.attrs and div.attrs['lang'] == 'srn':
            print(div)
    #print(div.attrs)

Issues:

- non-prose elements
- non-Dutch elements

In [19]:
import pandas as pd

ti_item_freq
vocab = list(item_freq.keys())
rows = []

for ti_id in ti_item_freq:
    row = [ti_id] + [ti_item_freq[ti_id][attr] if attr in ti_item_freq[ti_id] else 0 for attr in vocab]
    rows.append(row)


body_df = pd.DataFrame(rows, columns=['ti_id'] + vocab)
body_df.sum()

ti_id            stre009tele02hart029holl01
type--chapter                            34
type--section                             5
dtype: object

In [20]:
import pandas as pd

vocab = list(item_freq.keys())
rows = []

for ch_id in chapter_item_freq:
    row = [ch_id] + [chapter_item_freq[ch_id][attr] if attr in chapter_item_freq[ch_id] else 0 for attr in vocab]
    rows.append(row)


chapter_df = pd.DataFrame(rows, columns=['ch_id'] + vocab)
chapter_df.sum()

ch_id            stre009tele02-chapter-000stre009tele02-chapter...
type--chapter                                                    0
type--section                                                    5
dtype: object

In [21]:
#df['type--chapter'].min()
#df.sum()

In [23]:
chapter_df.columns

Index(['ch_id', 'type--chapter', 'type--section'], dtype='object')

In [24]:
body = soup.find('body')
children = [child for child in body if child.name is not None]
children[0]
[child.attrs for child in children]
item_freq.update([item for div in body.find_all('div') for item in div.attrs.items()])


In [25]:
import spacy


nlp_full = spacy.load("nl_core_news_md")

# Load the pipeline without the entity recognizer
nlp_small = spacy.load("nl_core_news_md", exclude=["ner", "parser"])
nlp_small.add_pipe("sentencizer")

In [83]:
%%time
doc_full = nlp_full(dutch_prose_text)


CPU times: user 19.5 s, sys: 878 ms, total: 20.3 s
Wall time: 21 s


In [84]:
%%time
doc_small = nlp_small(dutch_prose_text)


CPU times: user 9.37 s, sys: 299 ms, total: 9.67 s
Wall time: 9.72 s


In [85]:
nlp_small.max_length = 10_000_000
nlp_small.max_length

10000000

In [133]:
stats_rows = []

out_file = '../data/dbnl-chunks-tokenised-skip_first_200.tsv.gz'

headers = ['title_id', 'chunk_id', 'term', 'pos', 'term_freq']

with gzip.open(out_file, 'wt') as fh_out:
    header_string = '\t'.join(headers)
    fh_out.write(f"{header_string}\n")
    for ti, ti_id in enumerate(tei_file_map):
        with open(tei_file_map[ti_id], 'rt') as fh_in:
            soup = BeautifulSoup(fh_in, features='xml')
            dutch_prose_text = extract_dutch_prose_text(soup)
            doc = nlp_small(dutch_prose_text)
            chunks = chunk_doc(doc, skip_first=200)
            for row in make_chunk_rows(ti_id, chunks):
                row_string = '\t'.join([str(cell) for cell in row])
                fh_out.write(f"{row_string}\n")
        print(f"{ti+1} books parsed")

1 books parsed
2 books parsed
3 books parsed
4 books parsed
5 books parsed
6 books parsed
7 books parsed
8 books parsed
9 books parsed
10 books parsed
11 books parsed
12 books parsed
13 books parsed
14 books parsed
15 books parsed
16 books parsed
17 books parsed
18 books parsed
19 books parsed
20 books parsed
21 books parsed
22 books parsed
23 books parsed
24 books parsed
25 books parsed
26 books parsed
27 books parsed
28 books parsed
29 books parsed
30 books parsed
31 books parsed
32 books parsed
33 books parsed
34 books parsed
35 books parsed
36 books parsed
37 books parsed
38 books parsed
39 books parsed
40 books parsed
41 books parsed
42 books parsed
43 books parsed
44 books parsed
45 books parsed
46 books parsed
47 books parsed
48 books parsed
49 books parsed
50 books parsed
51 books parsed
52 books parsed
53 books parsed
54 books parsed
55 books parsed
56 books parsed
57 books parsed
58 books parsed
59 books parsed
60 books parsed
61 books parsed
62 books parsed
63 books parsed
6

## Filter Output by Available Titles

(2025-04-09: this only needs to be done because we have filtered more problematic (non-fiction) titles from the dataset)

In [31]:
import datetime

today = datetime.date.today().isoformat()

In [41]:
import gzip


orig_file = '../data/dbnl-chunks-tokenised-skip_first_200.tsv.gz'
new_file = f'../data/dbnl-chunks-tokenised-skip_first_200-version-{today}.tsv.gz'

len(title_ids)

with gzip.open(orig_file, 'rt') as fh_in:
    with gzip.open(new_file, 'wt') as fh_out:
        header_line = next(fh_in)
        fh_out.write(header_line)
        for line in fh_in:
            title_id, *rest = line.split('\t')
            if title_id in title_ids:
                fh_out.write(line)
            else:
                continue
                print(title_id, title_id in title_ids, rest)
                break

In [159]:
chapters_texts = extract_chapter_texts(soup)
for chapter_texts in chapters_texts:
    words = [word for text in chapter_texts for word in re.split(r"\s+", text)]
    print(len(words))

16
4873
7362
8645
12650
13468
11828
8234
18208
14602
12235
4630
3373


In [159]:
df = Counter()

prev_chunk = None
terms = set()
with gzip.open(out_file, 'rt') as fh:
    headers = next(fh).strip('\n').split('\t')
    for line in fh:
        cells = line.strip('\n').split('\t')
        row = {header: cells[hi] for hi, header in enumerate(headers)}
        if prev_chunk and prev_chunk != row['chunk_id']:
            df.update(terms)
            terms = set()
            
        #df.update([cells[2]])
        terms.add(row['term'])
        #print(cells)
        prev_chunk = row['chunk_id']


df.most_common(10)


[('en', 29202),
 ('de', 29161),
 ('zijn', 29133),
 ('in', 29108),
 ('het', 29055),
 ('op', 29053),
 ('dat', 29018),
 ('te', 29017),
 ('die', 28991),
 ('al', 28905)]

In [161]:
len(df), sum(df.values())

(1605779, 38137614)

In [183]:
num_chunks = 29260
max_df = num_chunks / 2
max_df

14630.0

In [184]:
rank = 0
for term, freq in df.most_common():
    rank += 1
    if freq < max_df:
        print(rank, term, freq)
        break
    

337 stoel 14628


In [100]:
from frog import Frog, FrogOptions


frog = Frog(FrogOptions(parser=False, ner=False, chunking=False, morph=False))


20250319:174605:662:requested configuration: /Users/marijnkoolen/.config/frog/nld/frog.cfg
20250319:174605:662:config read from: /Users/marijnkoolen/.config/frog/nld/frog.cfg
20250319:174605:662:configuration version = 0.12
20250319:174605:662:SET config nld
20250319:174605:662: MWU disabled, because the Parser is deselected
20250319:174605:662:tok-:Language List =[nld]
20250319:174605:662:tok-:rulesName=tokconfig-nld
20250319:174605:663:tok-:init tokenizer for languages: [nld]
20250319:174605:672:mblem-:Initiating lemmatizer...
20250319:174605:882:mbma-:Initiating morphological analyzer...
20250319:174607:668:tagger-tagger-:reading subsets from /Users/marijnkoolen/.config/frog/nld//subsets.cgn
20250319:174607:668:tagger-tagger-:reading constraints from /Users/marijnkoolen/.config/frog/nld//constraints.cgn
20250319:174611:972:NER-tagger-:READ  /Users/marijnkoolen/.config/frog/nld//ners.known
20250319:174612:300:NER-tagger-:loaded 13 additional Named Entities files
20250319:174612:300:W

In [101]:
#output = frog.process_raw("Dit is een test")
#print("RAW OUTPUT=",output)
output = frog.process("Dit is nog een test.")
#print("PARSED OUTPUT=",output)
len(output)
for token in output:
    print(token)

{'index': '1', 'text': 'Dit', 'lemma': 'dit', 'morph': '[dit]', 'pos': 'VNW(aanw,pron,stan,vol,3o,ev)', 'posprob': 0.777085, 'ner': 'O', 'chunker': 'B-NP'}
{'index': '2', 'text': 'is', 'lemma': 'zijn', 'morph': '[zijn]', 'pos': 'WW(pv,tgw,ev)', 'posprob': 0.999966, 'ner': 'O', 'chunker': 'B-VP'}
{'index': '3', 'text': 'nog', 'lemma': 'nog', 'morph': '[nog]', 'pos': 'BW()', 'posprob': 0.99982, 'ner': 'O', 'chunker': 'B-NP'}
{'index': '4', 'text': 'een', 'lemma': 'een', 'morph': '[een]', 'pos': 'LID(onbep,stan,agr)', 'posprob': 0.995781, 'ner': 'O', 'chunker': 'I-NP'}
{'index': '5', 'text': 'test', 'lemma': 'test', 'morph': '[test]', 'pos': 'N(soort,ev,basis,zijd,stan)', 'posprob': 0.903055, 'ner': 'O', 'chunker': 'I-NP'}
{'index': '6', 'text': '.', 'lemma': '.', 'morph': '[.]', 'pos': 'LET()', 'posprob': 1.0, 'ner': 'O', 'chunker': 'O', 'eos': True}


In [102]:
import time

start = time.time()
output = frog.process(dutch_prose_text)
step = time.time()
took = step - start
took

848.5281300544739

In [103]:
start = time.time()
doc = nlp_small(dutch_prose_text)
step = time.time()
took = step - start
took

0.8422760963439941

In [143]:
len(output)

print(' '.join([t['text'] for t in output[:200]]))

print(' '.join([t['chunker'] for t in output[:200]]))

output[197]

Bij de hoofdstuktitels is een punt toegevoegd als deze ontbrak in het origineel . p. 49 : de kop ‘ Nawoord ’ is tussen vierkante haken toegevoegd . 2017 dbnl / erven Felix Timmermans Felix Timmermans , De zeer schone uren van Juffrouw Symforosa , begijntjen . Den Nieuwe Boekhandel , Brussel / C.A.J. Van Dishoeck , Bussum 1918 Wijze van coderen : standaard De zeer schoone uren van Juffrouw Symforosa , Begijntjen DOOR FELIX TIMMERMANS UITGEGEVEN DOOR ‘ DEN NIEUWEN BOEKHANDEL ’ te BRUSSEL en C . -A . -J . VAN DISHOECK , te BUSSUM in 1918 . De lucht is natzilver als de rug van visch . Die dagen zijn er schoone wolken van allerhande gedaanten en verwen , en terwijl hier de zon een vinger licht door het venster steekt , staat er ginder boven de velden een helder stuksken regenboog . 't Is nu tijd om den tuin te bewerken . En 't begijntje Symforosa heeft den hovenier Martienus , die een neef is van mijnheer den onderpastoor naar haar hofken laten komen , om de groenten bedden op te dekken , b

{'index': '42',
 'text': '.',
 'lemma': '.',
 'morph': '[.]',
 'pos': 'LET()',
 'posprob': 1.0,
 'ner': 'O',
 'chunker': 'O',
 'eos': True}

In [146]:
frog_1 = Frog(FrogOptions(parser=False, ner=False, chunking=False, morph=False))

start = time.time()
output = frog_1.process(dutch_prose_text)
step = time.time()
took = step - start
took

20250320:162804:260:requested configuration: /Users/marijnkoolen/.config/frog/nld/frog.cfg
20250320:162804:260:config read from: /Users/marijnkoolen/.config/frog/nld/frog.cfg
20250320:162804:261:configuration version = 0.12
20250320:162804:261:SET config nld
20250320:162804:261: MWU disabled, because the Parser is deselected
20250320:162804:261:tok-:Language List =[nld]
20250320:162804:261:tok-:rulesName=tokconfig-nld
20250320:162804:263:tok-:init tokenizer for languages: [nld]
20250320:162804:276:mblem-:Initiating lemmatizer...
20250320:162806:240:tagger-tagger-:reading subsets from /Users/marijnkoolen/.config/frog/nld//subsets.cgn
20250320:162806:241:tagger-tagger-:reading constraints from /Users/marijnkoolen/.config/frog/nld//constraints.cgn
20250320:162806:241:Thu Mar 20 16:28:06 2025 Initialization done.


4.173191070556641

In [158]:
frog_4 = Frog(FrogOptions(parser=False, ner=False, chunking=False, morph=False, numThreads=b"4"))

start = time.time()
output = frog_4.process(dutch_prose_text)
step = time.time()
took = step - start
took

20250320:164157:335:requested configuration: /Users/marijnkoolen/.config/frog/nld/frog.cfg
20250320:164157:335:config read from: /Users/marijnkoolen/.config/frog/nld/frog.cfg
20250320:164157:335:configuration version = 0.12
20250320:164157:335:SET config nld
20250320:164157:335: MWU disabled, because the Parser is deselected
20250320:164157:335:WARNING!
---> There is NO OpenMP support enabled
---> --threads=4 is ignored.
---> Will continue on just 1 thread.
20250320:164157:335:tok-:Language List =[nld]
20250320:164157:335:tok-:rulesName=tokconfig-nld
20250320:164157:336:tok-:init tokenizer for languages: [nld]
20250320:164157:347:mblem-:Initiating lemmatizer...
20250320:164159:241:tagger-tagger-:reading subsets from /Users/marijnkoolen/.config/frog/nld//subsets.cgn
20250320:164159:241:tagger-tagger-:reading constraints from /Users/marijnkoolen/.config/frog/nld//constraints.cgn
20250320:164159:241:Thu Mar 20 16:41:59 2025 Initialization done.


4.184182167053223

## Testing lemmatisation of older spelling

In [5]:
import glob
import os

from bs4 import BeautifulSoup

from tei_parsing import extract_dutch_prose_text

In [6]:
tei_dir = '../../../../Data/DBNL/xml/1800-2000/xml Proza 19e en 20e eeuw algemeen DBNL 20250122/'
tei_files = glob.glob(os.path.join(tei_dir, '*'))
tei_file = [t for t in tei_files if 'kist001' in t][0]
with open(tei_file, 'rt') as fh:
    soup = BeautifulSoup(fh, features='xml')
    dutch_prose_text = extract_dutch_prose_text(soup)

In [7]:
from frog import Frog, FrogOptions

frog = Frog(FrogOptions(parser=False, ner=False, chunking=False, morph=False))

20250325:095944:266:requested configuration: /Users/marijnkoolen/.config/frog/nld/frog.cfg
20250325:095944:266:config read from: /Users/marijnkoolen/.config/frog/nld/frog.cfg
20250325:095944:267:configuration version = 0.12
20250325:095944:267:SET config nld
20250325:095944:267: MWU disabled, because the Parser is deselected
20250325:095944:268:tok-:Language List =[nld]
20250325:095944:268:tok-:rulesName=tokconfig-nld
20250325:095944:270:tok-:init tokenizer for languages: [nld]
20250325:095944:288:mblem-:Initiating lemmatizer...
20250325:095946:120:tagger-tagger-:reading subsets from /Users/marijnkoolen/.config/frog/nld//subsets.cgn
20250325:095946:121:tagger-tagger-:reading constraints from /Users/marijnkoolen/.config/frog/nld//constraints.cgn
20250325:095946:121:Tue Mar 25 09:59:46 2025 Initialization done.


In [8]:
len(dutch_prose_text)
output = frog.process(dutch_prose_text)

In [33]:
sent_old = "Vijf-en-twintig jaren zijn verloopen sedert tallooze tegenstrijdigheden in Chateaubriand's letterkundige nalatenschap."
sent_modern = "Vijf-en-twintig jaren zijn verlopen sedert talloze tegenstrijdigheden in Chateaubriand's letterkundige nalatenschap."

sent_out = frog.process(sent_old)
sent_out

[{'index': '1',
  'text': 'Vijf-en-twintig',
  'lemma': 'vijf-en-twintig',
  'pos': 'TW(hoofd,prenom,stan)',
  'posprob': 0.681935},
 {'index': '2',
  'text': 'jaren',
  'lemma': 'jaar',
  'pos': 'N(soort,mv,basis)',
  'posprob': 0.997633},
 {'index': '3',
  'text': 'zijn',
  'lemma': 'zijn',
  'pos': 'WW(pv,tgw,mv)',
  'posprob': 0.562801},
 {'index': '4',
  'text': 'verloopen',
  'lemma': 'verloopen',
  'pos': 'N(soort,mv,basis)',
  'posprob': 1.0},
 {'index': '5',
  'text': 'sedert',
  'lemma': 'sedert',
  'pos': 'VZ(init)',
  'posprob': 0.997963},
 {'index': '6',
  'text': 'tallooze',
  'lemma': 'tallooze',
  'pos': 'ADJ(prenom,basis,met-e,stan)',
  'posprob': 1.0},
 {'index': '7',
  'text': 'tegenstrijdigheden',
  'lemma': 'tegenstrijdigheid',
  'pos': 'N(soort,mv,basis)',
  'posprob': 0.999494},
 {'index': '8',
  'text': 'in',
  'lemma': 'in',
  'pos': 'VZ(init)',
  'posprob': 0.931126},
 {'index': '9',
  'text': "Chateaubriand's",
  'lemma': 'Chateaubriand',
  'pos': 'SPEC(deele

In [35]:
import spacy


#nlp_small = spacy.load("nl_core_news_md", exclude=["ner", "parser"])
#nlp_small.add_pipe("sentencizer")
doc = nlp_small(sent_old)
print(f'------------\nOLD: {sent_old}\n------------')
for spacy_sent in doc.sents:
    for token in spacy_sent:
        print([token.text, token.lemma_, token.pos_])


doc = nlp_small(sent_modern)
print(f'------------\nMODERN: {sent_modern}\n------------')
for spacy_sent in doc.sents:
    for token in spacy_sent:
        print([token.text, token.lemma_, token.pos_])




------------
OLD: Vijf-en-twintig jaren zijn verloopen sedert tallooze tegenstrijdigheden in Chateaubriand's letterkundige nalatenschap.
------------
['Vijf-en-twintig', 'Vijf-en-twintig', 'ADJ']
['jaren', 'jaar', 'NOUN']
['zijn', 'zijn', 'AUX']
['verloopen', 'verloopen', 'VERB']
['sedert', 'sedert', 'ADP']
['tallooze', 'tallooz', 'ADJ']
['tegenstrijdigheden', 'tegenstrijdighed', 'NOUN']
['in', 'in', 'ADP']
["Chateaubriand's", 'Chateaubriand', 'PROPN']
['letterkundige', 'letterkundig', 'ADJ']
['nalatenschap', 'nalatenschap', 'NOUN']
['.', '.', 'PUNCT']
------------
MODERN: Vijf-en-twintig jaren zijn verlopen sedert talloze tegenstrijdigheden in Chateaubriand's letterkundige nalatenschap.
------------
['Vijf-en-twintig', 'Vijf-en-twintig', 'ADJ']
['jaren', 'jaar', 'NOUN']
['zijn', 'zijn', 'AUX']
['verlopen', 'verlopen', 'VERB']
['sedert', 'sedert', 'ADP']
['talloze', 'talloos', 'ADJ']
['tegenstrijdigheden', 'tegenstrijdighed', 'NOUN']
['in', 'in', 'ADP']
["Chateaubriand's", 'Chateaubria

In [14]:
def has_long_vowel(term):
    long_vowels = ['ee', 'aa', 'oo', 'uu']
    return any([lv in term for lv in long_vowels])


[t['text'] for ti, t in enumerate(output[:800]) if has_long_vowel(t['text'])]

['exemplaar',
 'universiteitsbibliotheek',
 'signatuur',
 'een',
 'aantal',
 'een',
 'weergave',
 'twee',
 'naar',
 'een',
 'aantal',
 'tekstgedeelten',
 'origineel',
 'voorkomen',
 'maar',
 'eerste',
 'deel',
 'tweede',
 'deel',
 'tweede',
 'deel',
 'eerste',
 'deel',
 'vertaald',
 'Hoofdst',
 'Hoofdst',
 'Hoofdst',
 'zaaymannius',
 'een',
 'Hoofdst',
 'draaven',
 'naar',
 'Kasteel',
 'Hoofdst',
 'Zamenspraak',
 'Doop',
 'Hoofdst',
 'Zeldzaam',
 'voorval',
 'Hoofdst',
 'Hoofdst',
 'haaren',
 'Hoofdst',
 'Hoofdst',
 'Vroedmeester',
 'Hoofdst',
 'gebooren',
 'Hoofdst',
 'Hoofdst',
 'Hoofdst',
 'eenen',
 'Hoofdst',
 'Hoofdst',
 'Staatige',
 'naar',
 'Hoofdst',
 'Dorpsbestuur',
 'Hoofdst',
 'oorsprong',
 'Hoofdst',
 'Klooster',
 'Hoofdst',
 'Hoofdst',
 'Leeuw',
 'Hoofdst',
 'Droom',
 'Hoofdst',
 'Hoofdst',
 'Hoofdst',
 'Hoofdst',
 'Hoofdst',
 'Hoofdst',
 'oorsprong',
 'begraafenissen',
 'Hoofdst',
 'Hoofdst',
 'Vierschaar',
 'Inquisitie-Raad',
 'Hoofdst',
 'Hoofdst',
 'Hoofdst',
 'een',
 